In [7]:
import sqlite3

from pathlib import Path
from datetime import datetime

from bs4 import BeautifulSoup, SoupStrainer

In [8]:
con = sqlite3.connect("meter.db")

sql_create = """
CREATE TABLE IF NOT EXISTS extra (
    id INTEGER PRIMARY KEY,
    language TEXT,
    text TEXT
);

CREATE TABLE IF NOT EXISTS extra_ground_truth (
    ida INTEGER NOT NULL,
    idb INTEGER NOT NULL,
    label TEXT,
    FOREIGN KEY(ida) REFERENCES extra(id)
    FOREIGN KEY(idb) REFERENCES extra(id)
    PRIMARY KEY (ida, idb)
);

CREATE TABLE IF NOT EXISTS extra_predictions (
    ida INTEGER NOT NULL,
    idb INTEGER NOT NULL,
    scores TEXT,
    label TEXT,
    FOREIGN KEY(ida) REFERENCES extra(id)
    FOREIGN KEY(idb) REFERENCES extra(id)
    PRIMARY KEY (ida, idb)
);
"""

cur = con.cursor()

cur.executescript(sql_create)

In [9]:
bnc_base = Path("../ressourcen/BNC/Texts")
filelist = bnc_base / "news.txt"

extra_id = 0

for line in filelist.open():
    p = bnc_base / line.strip()
    
    with p.open(encoding="utf-8") as f:
        soup = BeautifulSoup(f)

    text = soup.find("wtext").text

    cur.execute(
        "INSERT OR IGNORE INTO extra VALUES (?, ?, ?)",
        (
            extra_id,
            "en",
            text.strip()
        )
    )

    extra_id += 1

In [10]:
con.commit()
con.close()